In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Data Governance Validation in BigQuery

## Overview
This notebook demonstrates how to verify **Data Governance** topics related to **BigQuery** Data in GCP by checking IAM permissions and Policies, and **Dataplex Universal Catalog** resources that exists in the project defined in the parameter **PROJECT_ID**.

**Dataplex Universal Catalog** replaces **Data Catalog**, providing more complex metadata management, advanced data governance features, more powerful data search, less complex access control, and a unified API to ease development.

The **Dataplex Universal Catalog** resources currently supported are: **Business Glossary (Glossary, GlossaryCategory, GlossaryTerm), and Entry Groups, Aspect Types, Entry Types and Entry(s)**.

Finally, this notebook showcases how to leverage `bigframes` to perform basic data quality tasks directly within BigQuery by using SQL.

**Data quality** is a critical aspect of any data-driven initiative, especially in machine learning and analytics. Poor data quality can lead to inaccurate insights, flawed models, and unreliable decisions. This notebook showcases a customized and manual approach to:

1.  **Data Profiling**: Understanding the structure, content, and quality of your data through descriptive statistics.
2.  **Anomaly Detection**: Identifying unusual or suspicious data points that deviate from expected patterns.

However it is strongly recommended that you can start using **Data Quality** and **Data Profling** Scans to avoid the use of extensive Python and SQL programming. **Dataplex Universal Catalog** lets you better understand the profile of your data by creating a **data profile scan**. A **data profile scan** is a type of **Dataplex Universal Catalog data scan** that analyzes a BigQuery table to generate statistical insights.

A **Dataplex Universal Catalog data scan** can be any of the following types:

1. **DATA_SCAN_TYPE_UNSPECIFIED:** The data scan type is unspecified.
2. **DATA_QUALITY:** Data quality scan.
3. **DATA_PROFILE:** Data profile scan.
4. **DATA_DISCOVERY:** Data discovery scan.
5. **DATA_DOCUMENTATION:** Data documentation scan.

## Contributors:

**HAZBLEYDI CERVERA**, LATAM Data Analytics Regional Team, hazbleydi@google.com

# Environment setup

Complete the tasks in this section to set up your environment.

**Please carefully review every single cell before run it and provide the required parameters if apply**

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

3. [Click here](https://console.cloud.google.com/flows/enableapi?apiid=bigquery.googleapis.com,bigqueryconnection.googleapis.com,cloudfunctions.googleapis.com,run.googleapis.com,artifactregistry.googleapis.com,cloudbuild.googleapis.com,cloudresourcemanager.googleapis.com) to enable the following APIs:

  * BigQuery API
  * BigQuery Connection API

4. If you are running this notebook locally, install the [Cloud SDK](https://cloud.google.com/sdk).

#### Set the PARAMETERS for the Notebook

If you don't know your project ID, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113).


In [ ]:
# dataframe:
# uuid: CF01CDDB-7A6B-4787-96BD-2FF7F493A261
# output_variable:
# config_str:

import google.colabsqlviz.explore_dataframe as _vizcell
_vizcell.explore_dataframe(df_or_df_name='', uuid='CF01CDDB-7A6B-4787-96BD-2FF7F493A261')

In [91]:
PROJECT_ID = "prj-dataplex-hcv"  # @param {type:"string"}
LOCATION = "US"
DATASET_ID = "imdb_raw"

# Set the project id
! gcloud config set project {PROJECT_ID}

Updated property [core/project].


#Installation

In [20]:
!pip install bigframes  --quiet
!pip install google-cloud-datacatalog  --quiet
!pip install --upgrade google-cloud-bigquery-datapolicies --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.6/226.6 kB 6.1 MB/s eta 0:00:00


### Colab only

Uncomment and run the following cell to restart the kernel:

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

Import Libraries

In [92]:
import bigframes.pandas as bf
import bigframes.bigquery as bq
from google.cloud import bigquery  # Still useful for some direct BQ client operations like INFORMATION_SCHEMA
import pandas as pd  # For local display if needed
import datetime # For timestamp comparisons

# IAM Policy Verification
 This script checks the IAM policy for a specified BigQuery dataset, flagging if any group or user has an overly permissive role like **'roles/bigquery.admin' or 'roles/bigquery.dataOwner'**. IAM Policy Verification (Dataset Level)

## Get the content of a Dataset in BigQuery
List of the tables that a dataset contains.
The dataset is referenced in the variable: **DATASET_ID**

In [50]:

# Construct a BigQuery client object.
client = bigquery.Client()

# TODO(developer): Set dataset_id to the ID of the dataset to fetch.
dataset_id = f"{PROJECT_ID}.{DATASET_ID}"

dataset = client.get_dataset(dataset_id)  # Make an API request.

full_dataset_id = "{}.{}".format(dataset.project, dataset.dataset_id)
friendly_name = dataset.friendly_name
print(
    "Got dataset '{}' with friendly_name '{}'.".format(
        full_dataset_id, friendly_name
    )
)

# View dataset properties.
print("Description: {}".format(dataset.description))
print("Labels:")
labels = dataset.labels
if labels:
    for label, value in labels.items():
        print("\t{}: {}".format(label, value))
else:
    print("\tDataset has no labels defined.")

# View tables in dataset.
print("Tables:")
tables = list(client.list_tables(dataset))  # Make an API request(s).
if tables:
    for table in tables:
        print("\t{}".format(table.table_id))
else:
    print("\tThis dataset does not contain any tables.")

Got dataset 'prj-dataplex-hcv.imdb_raw' with friendly_name 'None'.
Description: None
Labels:
	goog-dataplex-project-id: prj-dataplex-hcv
	goog-dataplex-lake-id: movie-lake
	goog-dataplex-zone-id: imdb-raw-zone
	goog-dataplex-asset-id: raw-movie-dataset
Tables:
	episode
	names
	principals
	title_basics
	title_ratings
	top_rated_horror_movies
	top_rated_horror_movies_with_cast_crew_details
	top_rated_movies


## [Dataset] Get the IAM Policy of a Dataset
Checks a BigQuery dataset's Access Control List.  The dataset is defined previously with the parameter **DATASET_ID**.

You can provide access to a dataset by granting an **IAM principal** a predefined or custom role that determines what the principal can do with the dataset. This is also known as attaching an **allow policy** to a resource. After granting access, you can view the dataset's access controls, and you can revoke access to the dataset.

For more information on access controls for BigQuery Datasets, see here -> https://docs.cloud.google.com/bigquery/docs/control-access-to-resources-iam

In [199]:
from google.cloud.bigquery import AccessEntry

def verify_dataset_iam_policy(project_id: str, dataset_id: str):
    """
    Checks a dataset's Access Control List (ACL).
    """
    print(f"--- Checking ACL for Dataset: {dataset_id} ---")
    bq_client = bigquery.Client()
    dataset_ref = bq_client.dataset(dataset_id, project=project_id)

    try:
        dataset = bq_client.get_dataset(dataset_ref)
        access_entries = dataset.access_entries

        fail_count = 0

        for entry in access_entries:
            print(f"Access controls for dataset '{dataset_id}':")
            print(f"  Role: {entry.role}, Entity Type: {entry.entity_type}, Entity ID: {entry.entity_id}")


    except Exception as e:
        print(f"⚠️ ERROR: Could not retrieve dataset policy: {e}")

# Execute the function
verify_dataset_iam_policy(PROJECT_ID, DATASET_ID)

--- Checking ACL for Dataset: demo_dataset ---
Access controls for dataset 'demo_dataset':
  Role: WRITER, Entity Type: specialGroup, Entity ID: projectWriters
Access controls for dataset 'demo_dataset':
  Role: OWNER, Entity Type: specialGroup, Entity ID: projectOwners
Access controls for dataset 'demo_dataset':
  Role: OWNER, Entity Type: userByEmail, Entity ID: admin@hazbleydi.altostrat.com
Access controls for dataset 'demo_dataset':
  Role: READER, Entity Type: specialGroup, Entity ID: projectReaders


## [Table] Get the IAM Policy (privileges/roles) assigned to a BigQuery table
Retrieves and displays the **IAM policy (privileges/roles)** assigned to a BigQuery table, safely handling both simple and conditional bindings.

You need to specify the table as a parameter using the variable **TABLE_ID**.

In [54]:
### --- Configuration --- ###
# TODO(developer): Set TABLE_ID to the TABLE in the dataset to fetch.
TABLE_ID = "episode"

from google.cloud import bigquery
from google.api_core import exceptions as gcp_exceptions

# Function
def get_table_iam_policy(project_id: str, dataset_id: str, table_id: str):
    """
    Retrieves and displays the IAM policy (privileges/roles) assigned to a BigQuery table,
    safely handling both simple and conditional bindings.
    """

    print(f"--- Checking IAM Policy for Table: {project_id}.{dataset_id}.{table_id} ---")

    try:
        # Initialize the BigQuery client
        bq_client = bigquery.Client(project=project_id)

        # 1. Get the reference to the table
        table_ref = bq_client.dataset(dataset_id).table(table_id)

        # 2. Retrieve the IAM policy for the table resource
        policy = bq_client.get_iam_policy(table_ref)

        if not policy.bindings:
            print("✅ PASS: No explicit IAM roles are assigned directly to this table.")
            print("    (Privileges are likely inherited from the Dataset or Project level.)")
            return

        print(f"\nPolicy Version: {policy.version}, Etag: {policy.etag}\n")
        print("🛑 WARNING: Roles listed below are DIRECTLY assigned to the table.")
        print("------------------------------------------------------------------")

        # 3. Iterate through bindings (Role -> Members) and display privileges
        for binding in policy.bindings:

            # Safely get role, members, and condition regardless of whether
            # 'binding' is a Policy.Binding object or a raw dict (for conditional bindings).
            if isinstance(binding, dict):
                # Use safe dictionary access for conditional bindings
                role = binding.get('role')
                members = binding.get('members')
                condition = binding.get('condition', None)
            else:
                # Assume standard object structure (Policy.Binding) and use attribute access
                # NOTE: This is safe because if it fails, the object doesn't contain the expected data.
                role = getattr(binding, 'role', None)
                members = getattr(binding, 'members', [])
                condition = getattr(binding, 'condition', None)

            if not role:
                print(f"⚠️ UNEXPECTED BINDING STRUCTURE: Could not find 'role' in binding: {binding}")
                continue

            # The 'role' is the privilege assigned (e.g., 'roles/bigquery.dataViewer')

            print(f"▶️ **Role (Privilege):** **{role}**")
            print("   Assigned to Members:")

            for member in members:
                print(f"   - {member}")

            if condition:
                # Display the condition details if present (handles both dict and object structures)
                title = condition.get('title') if isinstance(condition, dict) else getattr(condition, 'title', 'N/A')
                expression = condition.get('expression') if isinstance(condition, dict) else getattr(condition, 'expression', 'N/A')
                print(f"   *Condition:* {title} (Expression: {expression})")

        print("\n------------------------------------------------------------------")

    except gcp_exceptions.NotFound:
        print(f"❌ ERROR: Table '{table_id}' not found in dataset '{dataset_id}'. Check the IDs.")
    except Exception as e:
        print(f"⚠️ ERROR: An unexpected error occurred while fetching the IAM policy: {e}")


# Execute the function
get_table_iam_policy(PROJECT_ID, DATASET_ID, TABLE_ID)

--- Checking IAM Policy for Table: prj-dataplex-hcv.imdb_raw.episode ---

Policy Version: 1, Etag: BwZFEr8q0A0=

🛑 WARNING: Roles listed below are DIRECTLY assigned to the table.
------------------------------------------------------------------
▶️ **Role (Privilege):** **roles/bigquery.filteredDataViewer**
   Assigned to Members:
   - user:hazbleydi@google.com

------------------------------------------------------------------


# Data Catalog Verification
 This script checks the **Dataplex Universal Catalog** resources that exists in the project defined in the parameter **PROJECT_ID**.

 **Dataplex Universal Catalog** replaces **Data Catalog**, providing more complex metadata management, advanced data governance features, more powerful data search, less complex access control, and a unified API to ease development.

 The **Dataplex Universal Catalog** resources currently supported are: **Business Glossary (Glossary, GlossaryCategory, GlossaryTerm), and Entry Groups, Aspect Types, Entry Types and Entry(s)**.

 For more information about **Dataplex Universal Catalog**, please see here -> https://docs.cloud.google.com/dataplex/docs/catalog-overview

In [55]:
# Calls a Google Cloud REST API using the current users credentials.
# This is a utility function, please don't modify it
def restAPIHelper(url: str, http_verb: str, request_body: str) -> str:
  """Calls the Google Cloud REST API passing in the current users credentials"""

  import google.auth.transport.requests
  import requests
  import google.auth
  import json

  # Get an access token based upon the current user
  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request()
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
    "Content-Type" : "application/json",
    "Authorization" : "Bearer " + access_token
  }

  if http_verb == "GET":
    response = requests.get(url, headers=headers)
  elif http_verb == "POST":
    response = requests.post(url, json=request_body, headers=headers)
  elif http_verb == "PUT":
    response = requests.put(url, json=request_body, headers=headers)
  elif http_verb == "PATCH":
    response = requests.patch(url, json=request_body, headers=headers)
  elif http_verb == "DELETE":
    response = requests.delete(url, headers=headers)
  else:
    raise RuntimeError(f"Unknown HTTP verb: {http_verb}")

  if response.status_code == 200:
    return json.loads(response.content)
    #image_data = json.loads(response.content)["predictions"][0]["bytesBase64Encoded"]
  else:
    error = f"Error restAPIHelper -> ' Status: '{response.status_code}' Text: '{response.text}'"
    raise RuntimeError(error)

In [56]:
# JSON utility function
# This is a utility function, please don't modify it
def PrettyPrintJson(json_string):
  json_object = json.loads(json_string)
  json_formatted_str = json.dumps(json_object, indent=2)
  return json_formatted_str

## List Business Glossaries
Lists the **Business Glossary** resources for the project **PROJECT_ID**.

**Dataplex Universal Catalog Business Glossary** helps streamline data discovery and reduce ambiguity, resulting in better governance, more accurate analysis, and faster insights

For more information about **Business Glossaries**, please see here -> https://docs.cloud.google.com/dataplex/docs/manage-glossaries

In [57]:
### --- Configuration --- ###
# TODO(developer): Set the variable GLOSSARY_LOCATION.  A glossary location can be different from the BigQuery dataset location.
GLOSSARY_LOCATION = "us"

# Function
def existsingGlossaries(parent_path: str):
  """Checks to see if any Glossary already exists"""

  # https://dataplex.googleapis.com/v1/{parent=projects/*/locations/*}/glossaries
  url = f"https://dataplex.googleapis.com/v1/{parent_path}/glossaries"

  json_result = restAPIHelper(url, "GET", None)

  total_glossaries = 0
  if "glossaries" in json_result:
      for item in json_result["glossaries"]:
          total_glossaries += 1
      print(f"existsGlossary (GET) json_result: {PrettyPrintJson(json.dumps(json_result))}")


  if total_glossaries > 0:
    print(f"There are {total_glossaries} business glossaries created in the project {PROJECT_ID}.")
  else:
    print("There are no business glossaries created in the project")

# Execute the function
existsingGlossaries(f"projects/{PROJECT_ID}/locations/{GLOSSARY_LOCATION}")

existsGlossary (GET) json_result: {
  "glossaries": [
    {
      "name": "projects/prj-dataplex-hcv/locations/us/glossaries/clfnonnmk-775383de-8101f98d-768e2fe2",
      "uid": "e4ecaa4c-887d-4146-bf1d-625649fc6207",
      "displayName": "glosario-1",
      "createTime": "2025-12-04T04:42:45.316804088Z",
      "updateTime": "2025-12-04T04:42:46.916365637Z",
      "termCount": 2,
      "categoryCount": 2,
      "etag": "EXA8-8si2WZSyADWAtN9mw1mXk_P0-kLz8MDAELN4Hk"
    },
    {
      "name": "projects/prj-dataplex-hcv/locations/us/glossaries/flgnijmmk-d2c1d5c4-941cdc36-ec618ab4",
      "uid": "50ba4239-79ef-4ab4-9bc6-b1ee297ec824",
      "displayName": "glosario-2-demo",
      "createTime": "2025-12-04T04:47:26.670286938Z",
      "updateTime": "2025-12-04T04:47:28.004315785Z",
      "termCount": 1,
      "categoryCount": 1,
      "etag": "l81oMal3T_1TVZE5hSS0Jap98vxjNSCLkD-2zbDJUGo"
    }
  ]
}
There are 2 business glossaries created in the project prj-dataplex-hcv.


## List Glossary Categories
Lists the detail of **Glossary categories** for the project **PROJECT_ID**.

A category lets you organize and structure various categories and terms. **Categories** are defined within a **business glossary**. You can nest categories up to three levels.

For more information about **Business Glossaries**, see here -> https://docs.cloud.google.com/dataplex/docs/manage-glossaries

In [58]:
### --- Configuration --- ###
# TODO(developer): Set the variable GLOSSARY_CATEGORY_LOCATION.  A glossary location can be different from the BigQuery dataset location.
# A glossary category has the same location as its parent glossary.
GLOSSARY_CATEGORY_LOCATION = "us"

# Function
def listGlossaryCategories(parent_path: str):
    """List the detail of the Glossary Categories if exists"""

    # 1. First, list all glossaries
    # API Documentation: https://dataplex.googleapis.com/v1/{parent=projects/*/locations/*}/glossaries
    url = f"https://dataplex.googleapis.com/v1/{parent_path}/glossaries"

    json_result = restAPIHelper(url, "GET", None)

    total_categories = 0
    if "glossaries" in json_result:
        for item in json_result["glossaries"]:

            full_glossary_name = item["name"]
            display_name = item.get("displayName", "N/A")
            print(f"👉 Found Glossary: **{display_name}**")

            # Extract the actual glossary ID from the end of the full name.
            # Example: 'projects/.../glossaries/my_glossary_1' -> 'my_glossary_1'
            glossary_id = full_glossary_name.split('/')[-1]

            # 2. Second, call the GetGlossary API to try and get category details
            # The correct endpoint for CATEGORIES would be /glossaries/{glossary_id}/categories

            url2 = f"https://dataplex.googleapis.com/v1/{parent_path}/glossaries/{glossary_id}/categories"

            # Fetch the categories:
            # Re-calculating the parent path for the Categories endpoint,
            # which is the full glossary name.
            categories_parent_path = full_glossary_name
            url_categories = f"https://dataplex.googleapis.com/v1/{categories_parent_path}/categories"

            json_result_categories = restAPIHelper(url_categories, "GET", None)

            # Check if categories exist in the response
            if "categories" in json_result_categories and len(json_result_categories["categories"]) > 0:
                 print(f"Existing Categories for Glossary {display_name} (GET) json_result: {PrettyPrintJson(json.dumps(json_result_categories))}")
                 total_categories += len(json_result_categories["categories"])
            else:
                 print(f"No Categories found for Glossary: {display_name}")

    if total_categories > 0:
        print(f"There are {total_categories} categories in total across all glossaries created in the project {PROJECT_ID}.")
    else:
        print("There are no categories in the glossaries created in the project.")

# Execute the function
listGlossaryCategories(f"projects/{PROJECT_ID}/locations/{GLOSSARY_CATEGORY_LOCATION}")


👉 Found Glossary: **glosario-1**
Existing Categories for Glossary glosario-1 (GET) json_result: {
  "categories": [
    {
      "name": "projects/prj-dataplex-hcv/locations/us/glossaries/clfnonnmk-775383de-8101f98d-768e2fe2/categories/jkpeoaigg-8234e3d9-ae1d4c1c-1db63885",
      "uid": "0b79b7d7-313f-4e2e-9052-2b4f72536899",
      "displayName": "Categoria-2",
      "createTime": "2025-12-04T04:43:43.281679Z",
      "updateTime": "2025-12-04T04:43:43.281679Z",
      "parent": "projects/prj-dataplex-hcv/locations/us/glossaries/clfnonnmk-775383de-8101f98d-768e2fe2"
    },
    {
      "name": "projects/prj-dataplex-hcv/locations/us/glossaries/clfnonnmk-775383de-8101f98d-768e2fe2/categories/cghfnckob-c072d593-8de33ce7-f61c1c97",
      "uid": "597204b2-e266-4ebf-8134-2b5b53b1dea2",
      "displayName": "Categoria-1",
      "createTime": "2025-12-04T04:43:15.311775Z",
      "updateTime": "2025-12-04T04:43:15.311775Z",
      "parent": "projects/prj-dataplex-hcv/locations/us/glossaries/clfnonn

## List Terms in a Glossary Category
Lists the detail of the terms that exist in a category of a glossary in the project **PROJECT_ID**.

A **term** (short for "glossary term") describes a concept that's used in a particular branch of business within your company. For example, the marketing department of a company might create a term that describes cost per click. **Terms** are defined within a **business glossary**, either in the glossary directly or within any category found in the glossary.

For more information about **Business Glossaries**, see here -> https://docs.cloud.google.com/dataplex/docs/manage-glossaries

In [59]:
### --- Configuration --- ###
# TODO(developer): Set the variable GLOSSARY_CATEGORY_TERMS_LOCATION.  A glossary location can be different from the BigQuery dataset location.
# All the terms in a glossary category has the same location as its parent glossary.
GLOSSARY_CATEGORY_TERMS_LOCATION = "us"

# Function
def listGlossaryCategoryTerms(parent_path: str):
    """
    Lists the glossaries, their categories, and their terms
    within the specified parent path (projects/*/locations/*).
    """

    # 1. API Call: List all glossaries
    url_glossaries = f"https://dataplex.googleapis.com/v1/{parent_path}/glossaries"
    json_result_glossaries = restAPIHelper(url_glossaries, "GET", None)

    total_categories = 0
    total_terms = 0

    if "glossaries" in json_result_glossaries:
        print("--- Found Glossaries ---")
        for item in json_result_glossaries["glossaries"]:

            # Extract Glossary Info
            display_name = item.get("displayName", "N/A")
            full_glossary_name = item["name"] # e.g., projects/.../glossaries/my_glossary_id
            glossary_id = full_glossary_name.split('/')[-1]

            print(f"\n✨ Glossary: **{display_name}** (ID: {glossary_id})")

            # --- API Call 2: List Categories for the current Glossary ---
            categories_parent_path = full_glossary_name
            url_categories = f"https://dataplex.googleapis.com/v1/{categories_parent_path}/categories"

            json_result_categories = restAPIHelper(url_categories, "GET", None)

            if "categories" in json_result_categories:
                 category_count = len(json_result_categories["categories"])
                 total_categories += category_count
                 print(f"  - Categories Found: {category_count}")
                 print(f"  - Categories Details: {PrettyPrintJson(json.dumps(json_result_categories))}")
            else:
                 print("  - Categories Found: 0")

            # List Terms for the current Glossary ---
            # The API path for terms uses the full glossary name as the parent.
            terms_parent_path = full_glossary_name
            url_terms = f"https://dataplex.googleapis.com/v1/{terms_parent_path}/terms"

            json_result_terms = restAPIHelper(url_terms, "GET", None)

            if "terms" in json_result_terms:
                 term_count = len(json_result_terms["terms"])
                 total_terms += term_count
                 print(f"  - Terms Found: {term_count}")
                 print(f"  - Terms Details: {PrettyPrintJson(json.dumps(json_result_terms))}")
            else:
                 print("  - Terms Found: 0")

        print("\n--- Summary ---")

    else:
        print(f"No glossaries found in the project: {parent_path}")


    if total_categories > 0 or total_terms > 0:
        print(f"Total Categories found across all glossaries: {total_categories}")
        print(f"Total Terms found across all glossaries: {total_terms}")
    else:
        print("There were no categories or terms found in any glossaries created in the project {PROJECT_ID}.")

# Execute the function
listGlossaryCategoryTerms(f"projects/{PROJECT_ID}/locations/{GLOSSARY_CATEGORY_TERMS_LOCATION}")

--- Found Glossaries ---

✨ Glossary: **glosario-1** (ID: clfnonnmk-775383de-8101f98d-768e2fe2)
  - Categories Found: 2
  - Categories Details: {
  "categories": [
    {
      "name": "projects/prj-dataplex-hcv/locations/us/glossaries/clfnonnmk-775383de-8101f98d-768e2fe2/categories/jkpeoaigg-8234e3d9-ae1d4c1c-1db63885",
      "uid": "0b79b7d7-313f-4e2e-9052-2b4f72536899",
      "displayName": "Categoria-2",
      "createTime": "2025-12-04T04:43:43.281679Z",
      "updateTime": "2025-12-04T04:43:43.281679Z",
      "parent": "projects/prj-dataplex-hcv/locations/us/glossaries/clfnonnmk-775383de-8101f98d-768e2fe2"
    },
    {
      "name": "projects/prj-dataplex-hcv/locations/us/glossaries/clfnonnmk-775383de-8101f98d-768e2fe2/categories/cghfnckob-c072d593-8de33ce7-f61c1c97",
      "uid": "597204b2-e266-4ebf-8134-2b5b53b1dea2",
      "displayName": "Categoria-1",
      "createTime": "2025-12-04T04:43:15.311775Z",
      "updateTime": "2025-12-04T04:43:15.311775Z",
      "parent": "projects/

## Metadata Management in Dataplex Universal Catalog
 **Dataplex Universal Catalog** provides a central platform to store, manage, and access the metadata of your GCP Project. The following sections describe the metadata management features of **Dataplex Universal Catalog**.

In [73]:
# Metadata management in Dataplex Universal Catalog
from IPython.display import Image

# Print the title/description
print("Aspect types and entry types relationship in Dataplex Universal Catalog\n")
print("For more information about Dataplex Universal Catalog, please see here -> https://docs.cloud.google.com/dataplex/docs/catalog-overview\n")
Image(url='https://docs.cloud.google.com/static/dataplex/images/aspect-entry-type.png', width=1200)

Aspect types and entry types relationship in Dataplex Universal Catalog

For more information about Dataplex Universal Catalog, please see here -> https://docs.cloud.google.com/dataplex/docs/catalog-overview



## List Aspect Types
Lists AspectType resources in a Data Catalog for the project **PROJECT_ID**.

An **aspect type** is a reusable template for aspects. An **aspect** is a set of related metadata fields that lets you capture metadata within entries to provide meaningful context. You can use aspects to store:

**Business metadata:** Information that provides business context, such as
data classification.

**Technical metadata:** Technical details about the data asset, for example, its schema.

**Data-derived metadata:** Information generated from the data itself, such as statistics from a BigQuery table.

For more information about **Aspect Types**, see here -> https://docs.cloud.google.com/dataplex/docs/enrich-entries-metadata#aspect-types



In [98]:
### --- Configuration --- ###
# TODO(developer): Set the variable aspectTypeLocation.
# An Aspect Type is a resource in a Data Catalog whose location can be different from the BigQuery dataset location.
aspectTypeLocation = "global"

# Function
def listsAspectType(parent_path: str):
    """Checks to see if an Aspect Type already exists"""

    url = f"https://dataplex.googleapis.com/v1/{parent_path}/aspectTypes"

    json_result = restAPIHelper(url, "GET", None)
    #print(f"existsAspectType (GET) json_result: {json_result}")
    total_aspectTypes = 0

    # Test to see if exists, if so return
    if "aspectTypes" in json_result:
        for item in json_result["aspectTypes"]:
            total_aspectTypes += 1
            aspectTypeId = item["name"].split("/")[-1]

        print(f"existsAspectType (GET) json_result: {PrettyPrintJson(json.dumps(json_result))}")

        if total_aspectTypes > 0:
            print(f"There are {total_aspectTypes} aspect type(s) created in the Data Catalog for the project {PROJECT_ID}.")
    else:
        print("There are no aspect types created in the Data Catalog of this location.")

# Execute the function
listsAspectType(f"projects/{PROJECT_ID}/locations/{aspectTypeLocation}")

There are no aspect types created in the Data Catalog of this location.


## List Entry Types
Lists **EntryType** resources in a Data Catalog for the project **PROJECT_ID**.

An **entry type** is a template for creating entries in the catalog. An entry represents a data asset, for example a BigQuery table named **test-project.sales_data.customer_orders** is represented as an entry.

For more information about **Entry Types**, please see here -> https://docs.cloud.google.com/dataplex/docs/ingest-custom-sources#entry-types

In [99]:
### --- Configuration --- ###
# TODO(developer): Set the variable entryTypeLocation.
# An Entry Type is a resource in a Data Catalog whose location can be different from the BigQuery dataset location.
entryTypeLocation = "global"

# Function
def listsEntryType(parent_path: str):
    """Checks to see if an Entry Type already exists"""

    # https://cloud.google.com/dataplex/docs/reference/rest/v1/projects.locations.entryTypes/list
    url = f"https://dataplex.googleapis.com/v1/{parent_path}/entryTypes"


    json_result = restAPIHelper(url, "GET", None)
    total_entryTypes = 0

    # Test to see if exists, if so return
    if "entryTypes" in json_result:
        for item in json_result["entryTypes"]:
            total_entryTypes += 1
            entryTypeId = item["name"].split("/")[-1]

        print(f"entryType (GET) json_result: {PrettyPrintJson(json.dumps(json_result))}")

        if total_entryTypes > 0:
            print(f"There are {total_entryTypes} entry type(s) created in the Data Catalog for the project {PROJECT_ID}.")
    else:
        print("There are no entry types created in the Data Catalog of this location.")

# Execute the function
listsEntryType(f"projects/{PROJECT_ID}/locations/{entryTypeLocation}")

There are no entry types created in the Data Catalog of this location.


## List Entry Groups
Lists **EntryGroups** resources in a Data Catalog for the project **PROJECT_ID**.

An **entry group** is a container for one or more entries. You can use entry groups to manage access control and regional location for the entries. Every **entry group** belongs to a project.

For more information about **entry groups**, please see here -> https://docs.cloud.google.com/dataplex/docs/ingest-custom-sources#entry-groups

In [100]:
### --- Configuration --- ###
# TODO(developer): Set the variable entryGroupLocation.
# An Entry Group is a resource in a Data Catalog whose location can be different from the BigQuery dataset location.
entryGroupLocation = "global"

# Function
def listsEntryGroups(parent_path: str):
    """Checks to see if an Entry Type already exists"""

    # https://cloud.google.com/dataplex/docs/reference/rest/v1/projects.locations.entryGroups/list
    url = f"https://dataplex.googleapis.com/v1/{parent_path}/entryGroups"

    json_result = restAPIHelper(url, "GET", None)
    total_entryGroups = 0

    # Test to see if exists, if so return
    if "entryGroups" in json_result:
        for item in json_result["entryGroups"]:
            total_entryGroups += 1
            entryGroupId = item["name"].split("/")[-1]

        print(f"entryGroup (GET) json_result: {PrettyPrintJson(json.dumps(json_result))}")

        if total_entryGroups > 0:
            print(f"There are {total_entryGroups} entry Group(s) created in the Data Catalog for the project {PROJECT_ID}.")
    else:
        print("There are no entry Groups created in the Data Catalog of this location.")

# Execute the function
listsEntryGroups(f"projects/{PROJECT_ID}/locations/{entryGroupLocation}")

entryGroup (GET) json_result: {
  "entryGroups": [
    {
      "name": "projects/prj-dataplex-hcv/locations/global/entryGroups/@analyticshub",
      "uid": "ab94135a-55c0-470b-86e3-8a91b224f439",
      "createTime": "2024-06-06T18:46:50.208486904Z",
      "updateTime": "2024-06-06T18:46:53.250786983Z",
      "etag": "R0AN6Bv9uLPHDyIEQC2z3lNZNONv1B9D4-3XQGH9Fec"
    },
    {
      "name": "projects/prj-dataplex-hcv/locations/global/entryGroups/@storage",
      "uid": "7df0b2a7-c78a-4143-bf4d-96d142cb120f",
      "createTime": "2024-06-07T12:24:53.281182442Z",
      "updateTime": "2024-06-07T12:24:56.467112441Z",
      "etag": "BI05kWpcRhWo1Htxo7SMWZzd3RHUbJqOFqKN_m1OQnQ"
    },
    {
      "name": "projects/prj-dataplex-hcv/locations/global/entryGroups/@dataprocmetastore",
      "uid": "beb0be60-9160-452e-8f2e-555edf6e61ac",
      "createTime": "2024-06-07T09:17:52.304669782Z",
      "updateTime": "2024-06-07T09:17:55.613838877Z",
      "etag": "47slZlbea4_Fp0SKgXkKf6ZbDkzvXrZDbnkqjSujp

# Data Profiling
 **Data profiling** is the process of examining the data available in an information source and collecting statistics and information about that data. It helps us understand the structure, content, and quality of our dataset.

 **This section also uses SQL to run a data profile for a table in BigQuery**, however for a deeper data profile analysis, it is **strongly recommended** to use the the **data profile scan module** available in the **Dataplex Universal Catalog**. Please see here -> https://docs.cloud.google.com/dataplex/docs/data-profiling-overview

## List Dataplex Universal Catalog Data Scans
Tests to see if a **Data Scan** exists in the project **PROJECT_ID**

**Dataplex Universal Catalog** lets you better understand the profile of your data by creating a **data profile scan**. A **data profile scan** is a type of **Dataplex Universal Catalog data scan** that analyzes a BigQuery table to generate statistical insights.

A **Dataplex Universal Catalog data scan** can be any of the following types:

*   **DATA_SCAN_TYPE_UNSPECIFIED:** The data scan type is unspecified.
*   **DATA_QUALITY:** Data quality scan.
*   **DATA_PROFILE:** Data profile scan.
*   **DATA_DISCOVERY:** Data discovery scan.
*   **DATA_DOCUMENTATION:** Data documentation scan.




In [104]:
# Metadata management in Dataplex Universal Catalog
from IPython.display import Image

# Print the title/description
print("The following diagram shows how Dataplex Universal Catalog scans data to report on statistical characteristics.")
print(">>> RECOMMENDED >>> Use the Data Profile Scan or the Data Quality scan Module in the Dataplex Universal Catalog\n")
print("please see here -> https://docs.cloud.google.com/dataplex/docs/data-profiling-overview\n")
Image(url='https://docs.cloud.google.com/static/dataplex/images/data-profile-scan.png', width=800)

The following diagram shows how Dataplex Universal Catalog scans data to report on statistical characteristics.
>>> RECOMMENDED >>> Use the Data Profile Scan or the Data Quality scan Module in Dataplex Universal Catalog

please see here -> https://docs.cloud.google.com/dataplex/docs/data-profiling-overview



In [184]:
### --- Configuration --- ###
# TODO(developer): Set the variable DATA SCAN regional location.
# A Data Scan is a regional resource in Dataplex whose location can be different from the BigQuery dataset location.
dataScanRegion = "us-central1"

# Function
def existsDataScan(parent_path: str):
  """Checks to see if a Data Scan of any type already exists"""

  # Gather existing data scans
  # https://cloud.google.com/dataplex/docs/reference/rest/v1/projects.locations.dataScans/list

  url = f"https://dataplex.googleapis.com/v1/{parent_path}/dataScans"

  # Gather existing data scans
  json_result = restAPIHelper(url, "GET", None)
  total_dataScans = 0

  # Test to see if data scan exists, if so return
  if "dataScans" in json_result:
    for item in json_result["dataScans"]:
      total_dataScans += 1
      dataScanId = item["name"].split("/")[-1]

    print(f"Data Scan(GET) json_result: {PrettyPrintJson(json.dumps(json_result))}")

    if total_dataScans > 0:
            print(f"\nThere are {total_dataScans} Data Scan(s) created in the project {PROJECT_ID}.")
  else:
      print("\nThere are no Data Scan(s) created in the project {PROJECT_ID}.")

# Execute the function
existsDataScan(f"projects/{PROJECT_ID}/locations/{dataScanRegion}")

Data Scan(GET) json_result: {
  "dataScans": [
    {
      "name": "projects/prj-dataplex-hcv/locations/us-central1/dataScans/ab3e23ec9-4182-4fbb-896c-f6502a74e87e",
      "uid": "6f7893a0-a2f6-4e51-a348-fbfb153ec84a",
      "description": "Profile scan for table \"\" with default configuration",
      "displayName": "-default-profile",
      "state": "ACTIVE",
      "createTime": "2025-03-20T17:01:56.591575279Z",
      "updateTime": "2025-03-20T17:02:01.957395870Z",
      "data": {
        "resource": "//bigquery.googleapis.com/projects/prj-dataplex-hcv/datasets/order_raw_zone/tables/order_raw_data"
      },
      "executionSpec": {
        "trigger": {
          "onDemand": {}
        }
      },
      "executionStatus": {
        "latestJobStartTime": "2025-03-20T17:02:37.679Z",
        "latestJobEndTime": "2025-03-20T17:03:05.786252574Z",
        "latestJobCreateTime": "2025-03-20T17:02:37.104428331Z"
      },
      "type": "DATA_PROFILE"
    },
    {
      "name": "projects/prj-dat

## Load a BigQuery table into a bigframe

This section uses **SQL** to run a manual and customized **data profile** for a table in BigQuery, however for a deeper data profile analysis, it is **strongly recommended** to use the the **data profile scan module** available in the **Dataplex Universal Catalog**.

For more information, please see here -> https://docs.cloud.google.com/dataplex/docs/data-profiling-overview

In [192]:
### --- Configuration --- ###
# TODO(developer): Set the TABLE_ID to run a manual SQL data profile for. The table should be part of the DATASET_ID defined at the beginning.
TABLE_ID = "customer_sample_data"

In [194]:
# Full table reference for `bigframes`
# *** PROJECT_ID, DATASET_ID, and TABLE_ID should be defined previously ***
FULL_TABLE_REF = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}"

#set project id using bigframe option
bf.options.bigquery.project=PROJECT_ID

# Initialize BigQuery client for direct SQL (e.g., INFORMATION_SCHEMA)
bq_client = bigquery.Client()

print(f"Connected to BigQuery via bigframes.")

# Load the BigQuery table into a bigframes DataFrame
try:
    df = bf.read_gbq_table(FULL_TABLE_REF)
    print(f"\nSuccessfully loaded BigQuery table {FULL_TABLE_REF} into bigframes DataFrame. Shape: {df.shape}")
except Exception as e:
    print(f"\nError loading table: {e}")
    print("\nPlease ensure the table {FULL_TABLE_REF} exists and you have the necessary permissions.")

Connected to BigQuery via bigframes.



Successfully loaded BigQuery table prj-dataplex-hcv.demo_dataset.customer_sample_data into bigframes DataFrame. Shape: (10000, 12)


## [SQL] Table Overview (Row Count, Column Names, Data Types)
We'll start by getting a high-level overview of the DataFrame, including its dimensions and column types.

In [195]:
print(f"\n--- Table Overview: [{FULL_TABLE_REF}] ---\n")
print(f"\nTotal rows: {df.shape[0]}")
print("\nColumn information:")
df.info()


--- Table Overview: [prj-dataplex-hcv.demo_dataset.customer_sample_data] ---


Total rows: 10000

Column information:
<class 'bigframes.dataframe.DataFrame'>


Index: 10000 entries, 0 to 9999
Data columns (total 12 columns):


  #  Column             Non-Null Count    Dtype
---  -----------------  ----------------  -------
  0  customer_id        10000 non-null    string
  1  first_name         10000 non-null    string
  2  last_name          9500 non-null     string
  3  email              9802 non-null     string
  4  phone_number       9500 non-null     string
  5  address            9500 non-null     string
  6  city               10000 non-null    string
  7  state              10000 non-null    string
  8  zip_code           10000 non-null    string
  9  registration_date  10000 non-null    string
 10  total_spent        9501 non-null     Float64
 11  customer_segment   10000 non-null    string
dtypes: Float64(1), string(11)


memory usage: 1040000 bytes


In [196]:
# Get descriptive statistics for numerical columns
print("Get descriptive statistics for numerical columns\n")
print(f"Table: [{FULL_TABLE_REF}]\n")
print(df.describe().to_pandas())

# Check for null values
print("Check for null values\n")
print(df.isnull().sum().to_pandas())

Get descriptive statistics for numerical columns

Table: [prj-dataplex-hcv.demo_dataset.customer_sample_data]



       total_spent
count       9501.0
mean   2564.907397
std    2617.505863
min       -4977.77
25%        1226.29
50%        2492.39
75%        3780.24
max       90953.88
Check for null values



customer_id            0
first_name             0
last_name            500
email                198
phone_number         500
address              500
city                   0
state                  0
zip_code               0
registration_date      0
total_spent          499
customer_segment       0
dtype: Int64


In [202]:
# Function to generate a customized Data Profile of a table in BigQuery by using SQL, Pandas and BigQuery Bigframe.
# For a deeper data profile analysis, it is strongly recommended to use the the data profile scan module available in the Dataplex Universal Catalog.
# Please see here for more information -> https://docs.cloud.google.com/dataplex/docs/data-profiling-overview

def profile_bigquery_table():
    """
    Connects to BigQuery, fetches data, and generates a detailed profile
    for each column, including statistical calculations and anomaly detection.

    Args:
        None

    Returns:
        dict: A dictionary containing the profiling results.
    """

    # 1. Setup BigQuery Client and Fetch Data
    client = bigquery.Client(project=PROJECT_ID)
    table_ref = f"{FULL_TABLE_REF}"
    print(f"Fetching data from table: {table_ref}")

    # Use a simple SELECT * to get all data (be mindful of large tables)
    query = f"SELECT * FROM `{table_ref}` LIMIT 100000" # Limit rows for stability

    # Fetch the data into a Pandas DataFrame
    df = client.query(query).to_dataframe()

    print(f"Data fetched. DataFrame shape: {df.shape}")

    profiling_results = {}

    # 2. Iterate Over Columns and Profile
    for column in df.columns:
        series = df[column]
        dtype = series.dtype
        results = {"column_name": column, "dtype": str(dtype)}

        # --- Profiling Numeric Columns ---
        if pd.api.types.is_numeric_dtype(series):
            # Calculate descriptive statistics
            results["type"] = "numeric"
            results["count"] = series.count()
            results["mean"] = series.mean()
            results["std_dev"] = series.std()
            results["min"] = series.min()
            results["max"] = series.max()
            results["quartiles"] = series.quantile([0.25, 0.5, 0.75]).to_dict()

            # Outlier Detection (Z-score)
            mean = series.mean()
            std = series.std()

            # Avoid division by zero for columns with constant values
            if std > 0:
                z_scores = (series - mean) / std
                # Identify outliers with |Z-score| > 3 (common threshold)
                outliers = series[np.abs(z_scores) > 3].dropna()
                results["outlier_count_z_score_3"] = len(outliers)
                results["outliers_sample"] = outliers.unique()[:10].tolist()
            else:
                results["outlier_count_z_score_3"] = 0
                results["outliers_sample"] = []

        # --- Profiling String Columns ---
        elif pd.api.types.is_object_dtype(series) or pd.api.types.is_string_dtype(series):
            results["type"] = "string"
            results["non_null_count"] = series.dropna().shape[0]
            results["distinct_count"] = series.nunique()

            # Top Occurring Values
            value_counts = series.value_counts().head(5)
            results["top_values"] = value_counts.to_dict()

            # Inconsistent String Value Identification (Rare Occurrences)
            # Find values that occur less than a low threshold (e.g., 2 times)
            rare_threshold = 2
            rare_values = value_counts[value_counts <= rare_threshold].index.tolist()
            results["rare_value_count"] = len(rare_values)
            results["rare_values_sample"] = rare_values[:10]

        # --- Profiling Date/Time Columns ---
        elif pd.api.types.is_datetime64_any_dtype(series) or pd.api.types.is_timedelta64_dtype(series):
            results["type"] = "datetime/timestamp"
            results["non_null_count"] = series.dropna().shape[0]
            results["min_date"] = series.min()
            results["max_date"] = series.max()

            if series.min() is not pd.NaT and series.max() is not pd.NaT:
                # Overall Time Range
                time_range = series.max() - series.min()
                results["overall_time_range"] = str(time_range)

                # Anomaly Detection (Dates far in the past/future)
                # Define a reasonable range (e.g., between 1900 and the current year)
                current_year = pd.Timestamp.now().year

                # Check for dates outside a sensible range (e.g., before 1900 or 10 years in future)
                anomaly_low = pd.to_datetime('1900-01-01')
                anomaly_high = pd.to_datetime(f'{current_year + 10}-12-31')

                anomalies = series[(series < anomaly_low) | (series > anomaly_high)].dropna()
                results["anomaly_count_range"] = len(anomalies)
                results["anomaly_sample"] = anomalies.unique()[:10].tolist()


        # --- Default/Other Types (e.g., Boolean, Array, Struct) ---
        else:
            results["type"] = "other"
            results["non_null_count"] = series.dropna().shape[0]
            results["distinct_count"] = series.nunique()

        profiling_results[column] = results

    return profiling_results

# --- Example Usage ---
profile_output = profile_bigquery_table()

print("\n*** SQL Data Profiling Results ***")

# # Print the results in a readable format
for col_name, profile in profile_output.items():
    print("\n")
    print("-" * 50)
    print(f"## 📊 Column: {col_name} (Type: {profile['type']})")
    for key, value in profile.items():
         if key not in ["column_name", "type"]:
             # Pretty print pandas series/timestamps
             if isinstance(value, pd.Timestamp):
                  print(f"  {key}: {value.isoformat()}")
             else:
                 print(f"  {key}: {value}")

Fetching data from table: prj-dataplex-hcv.demo_dataset.customer_sample_data
Data fetched. DataFrame shape: (10000, 12)

*** SQL Data Profiling Results ***


--------------------------------------------------
## 📊 Column: customer_id (Type: string)
  dtype: object
  non_null_count: 10000
  distinct_count: 9950
  top_values: {'c15c63a0-8f29-40d1-a4fe-5fcf38a38349': 2, 'b474de16-2aa8-4c0c-b0c6-7963d51ec771': 2, '9c345889-9b84-46f6-b0eb-8907ec975c75': 2, '0cb4d754-5711-487b-ad63-ddcff5ee39e4': 2, '3f575b38-b74d-45a2-b36d-4d887cfa5631': 2}
  rare_value_count: 5
  rare_values_sample: ['c15c63a0-8f29-40d1-a4fe-5fcf38a38349', 'b474de16-2aa8-4c0c-b0c6-7963d51ec771', '9c345889-9b84-46f6-b0eb-8907ec975c75', '0cb4d754-5711-487b-ad63-ddcff5ee39e4', '3f575b38-b74d-45a2-b36d-4d887cfa5631']


--------------------------------------------------
## 📊 Column: first_name (Type: string)
  dtype: object
  non_null_count: 10000
  distinct_count: 845
  top_values: {'Michael': 217, 'Jennifer': 159, 'James': 15

#Summary and next steps

This notebook demonstrates how to verify **Data Governance** topics related to **BigQuery** Data in GCP by checking **IAM permissions and Policies**, and **Dataplex Universal Catalog** resources that exists in the project defined in the parameter **PROJECT_ID**.

**Dataplex Universal Catalog** replaces **Data Catalog**, providing more complex metadata management, advanced data governance features, more powerful data search, less complex access control, and a unified API to ease development.

The **Dataplex Universal Catalog** resources currently supported are: **Business Glossary (Glossary, GlossaryCategory, GlossaryTerm), and Entry Groups, Aspect Types, Entry Types and Entry(s)**. Finally, this notebook showcased how to leverage `bigframes` to perform a customized data profiling tasks directly within BigQuery by using SQL.

As a next step, you can start using **Data Quality** and **Data Profling Scans** to avoid the use of extensive Python and SQL programming. **Dataplex Universal Catalog** lets you better understand the profile of your data by creating a **data profile scan**. A **data profile scan** is a type of **Dataplex Universal Catalog data scan** that analyzes a BigQuery table to generate statistical insights.

For more information, please see here -> https://docs.cloud.google.com/dataplex/docs/data-profiling-overview

